In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. Etiketli veriyi yükle
with open("etiketli_yorumlar_mistral.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data["reviews"])  # ["comment", "label"]

# 2. Özellik çıkarımı (TF-IDF)
vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1,2))
X = vectorizer.fit_transform(df["comment"])
y = df["label"]

# 3. Eğitim ve test bölmesi
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. Model eğitimi
model = LogisticRegression(max_iter=1000, class_weight="balanced")
model.fit(X_train, y_train)

# 5. Değerlendirme
y_pred = model.predict(X_test)
print("\n📊 Test Seti Değerlendirmesi:")
print(classification_report(y_test, y_pred, digits=3))


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(0)

In [ ]:
# 6. Yeni yorumları sınıflandır
def yorum_tahmin_et(model, vectorizer, yorumlar):
    X_yeni = vectorizer.transform(yorumlar)
    tahminler = model.predict(X_yeni)
    etiket_map = {-1: "Negatif", 0: "Nötr", 1: "Pozitif"}
    return [(yorum, etiket_map.get(t, "Bilinmiyor")) for yorum, t in zip(yorumlar, tahminler)]

# 7. Örnek test yorumları
yeni_yorumlar = [
    "Ürün gerçekten kaliteli ve şık",
    "Kargo geç geldi ama ürün güzel",
    "İade ettim çünkü çalışmıyordu",
    "Ne iyi ne kötü, idare eder"
]

sonuclar = yorum_tahmin_et(model, vectorizer, yeni_yorumlar)

print("\n🔍 Yeni Yorum Tahminleri:")
for yorum, etiket in sonuclar:
    print(f"'{yorum}' → {etiket}")
